<center><div style="direction:rtl;font-family:B Lotus, B Nazanin, Tahoma">به نام خدا</div></center>

<h1><center><div style="direction:rtl;font-family:B Lotus, B Nazanin, Tahoma">رگرسیون یا 
Regression داده های ساختار یافته
</div></center></h1>

## <div style="direction:rtl;text-align:right;font-family:B Lotus, B Nazanin, Tahoma">معرفی مجموعه داده تخممین قیمت منازل </div>

<div style="direction:rtl;text-align:right;font-family:Tahoma">
این مجموعه داده در مقاله سال 2016 تحت عنوان 2016
    House price estimation from visual and textual features
معرفی و منتشر شده است.
</div>
<p>
<a href="https://github.com/emanhamed/Houses-dataset" target="_blank">
https://github.com/emanhamed/Houses-dataset
</a>
<br>
<a href="https://arxiv.org/pdf/1609.08399.pdf" target="_blank">
https://arxiv.org/pdf/1609.08399.pdf
</a>
    
</p>

In [1]:
# import the necessary packages
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
import pandas as pd
import numpy as np
import glob
import cv2
import os
import locale

Using TensorFlow backend.


In [2]:
cols = ["bedrooms", "bathrooms", "area", "zipcode", "price"]
df = pd.read_csv("D:/dataset/Houses-dataset/Houses Dataset/HousesInfo.txt", sep=" ", header=None, names=cols)


In [3]:
df.head()

,bedrooms,bathrooms,area,zipcode,price
0,4,4.0,4053,85255,869500.0
1,4,3.0,3343,36372,865200.0
2,3,4.0,3923,85266,889000.0
3,5,5.0,4022,85262,910000.0
4,3,4.0,4116,85266,971226.0


<div style="direction:rtl;text-align:right;font-family:Tahoma">
کد پستی های منحصر به فرد  و تعدادشان
</div>

In [5]:
zipcodes, counts = np.unique(df["zipcode"], return_counts=True)
#dict(zip(zipcodes, counts))

In [6]:
df.shape

(535, 5)

<div style="direction:rtl;text-align:right;font-family:Tahoma">
    حذف رکوردهایی که zip code آن منزل ها zip code شان کمتر از 25 بار تکرا شده است
</div>

In [7]:
# loop over each of the unique zip codes and their corresponding
# count
for (zipcode, count) in zip(zipcodes, counts):
    # the zip code counts for our housing dataset is *extremely*
    # unbalanced (some only having 1 or 2 houses per zip code)
    # so let's sanitize our data by removing any houses with less
    # than 25 houses per zip code
    if count < 25:
        idxs = df[df["zipcode"] == zipcode].index
        df.drop(idxs, inplace=True)

In [8]:
df.shape

(362, 5)

<div style="direction:rtl;text-align:right;font-family:Tahoma">
جدا کردن داده های موجود به داده های آموزش و آزمون</div>

In [9]:
(train, test) = train_test_split(df, test_size=0.25, random_state=42)
print(train.shape)
print(test.shape)

(271, 5)
(91, 5)


## <div style="direction:rtl;text-align:right;font-family:B Lotus, B Nazanin, Tahoma">پیش پردازش داده ها:</div>

<div style="direction:rtl;text-align:right;font-family:Tahoma">
نرمال سازی داده ها یا Scaling  داده ها بین 0 و 1
</div>

In [10]:
# find the largest house price in the training set and use it to
# scale our house prices to the range [0, 1] (this will lead to
# better training and convergence)
maxPrice = train["price"].max()
trainY = train["price"] / maxPrice
testY = test["price"] / maxPrice

In [12]:
# initialize the column names of the continuous data
continuous = ["bedrooms", "bathrooms", "area"]

# performin min-max scaling each continuous feature column to
# the range [0, 1]
cs = MinMaxScaler()
trainContinuous = cs.fit_transform(train[continuous])
testContinuous = cs.transform(test[continuous])

C:\Users\alire\Miniconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


<div style="direction:rtl;text-align:right;font-family:Tahoma">
تبدیل مقادیر گسسته به one-hot</div>

In [14]:
# one-hot encode the zip code categorical data (by definition of
# one-hot encoing, all output features are now in the range [0, 1])
zipBinarizer = LabelBinarizer().fit(df["zipcode"])
trainCategorical = zipBinarizer.transform(train["zipcode"])
testCategorical = zipBinarizer.transform(test["zipcode"])

<div style="direction:rtl;text-align:right;font-family:Tahoma">
    ترکیب ویژگی‌های گستته و پیوسته
</div>

In [16]:
# construct our training and testing data points by concatenating
# the categorical features with the continuous features
trainX = np.hstack([trainCategorical, trainContinuous])
testX = np.hstack([testCategorical, testContinuous])

print(trainX.shape)
print(testX.shape)

(271, 10)
(91, 10)


## <div style="direction:rtl;text-align:right;font-family:B Lotus, B Nazanin, Tahoma">تعریف معماری مدل (model architecture)</div>

In [19]:
dim = trainX.shape[1]
# define our MLP network
model = Sequential()
model.add(Dense(8, input_dim=dim, activation="relu"))
model.add(Dense(4, activation="relu"))
model.add(Dense(1, activation="linear"))

## <div style="direction:rtl;text-align:right;font-family:B Lotus, B Nazanin, Tahoma">Compile model</div>

In [20]:
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)

## <div style="direction:rtl;text-align:right;font-family:B Lotus, B Nazanin, Tahoma">آموش مدل با داده‌های آموزشی</div>

In [21]:
model.fit(trainX, trainY, validation_data=(testX, testY),
    epochs=200, batch_size=8)

Train on 271 samples, validate on 91 samples
Epoch 1/200
271/271 [==============================] - 3s 11ms/step - loss: 274.3629 - val_loss: 109.6800
Epoch 2/200
271/271 [==============================] - 0s 852us/step - loss: 81.8187 - val_loss: 79.1573
Epoch 3/200
271/271 [==============================] - 0s 856us/step - loss: 57.5993 - val_loss: 63.2764
Epoch 4/200
271/271 [==============================] - 0s 915us/step - loss: 46.6169 - val_loss: 58.3518
Epoch 5/200
271/271 [==============================] - 0s 896us/step - loss: 46.4683 - val_loss: 55.3489
Epoch 6/200
271/271 [==============================] - 0s 815us/step - loss: 41.2374 - val_loss: 52.9153
Epoch 7/200
271/271 [==============================] - 0s 752us/step - loss: 41.2185 - val_loss: 50.9960
Epoch 8/200
271/271 [==============================] - 0s 749us/step - loss: 40.7949 - val_loss: 48.3711
Epoch 9/200
271/271 [==============================] - 0s 755us/step - loss: 37.9040 - val_loss: 47.3093
Epoch 10/

Epoch 78/200
271/271 [==============================] - 0s 730us/step - loss: 28.5560 - val_loss: 34.8651
Epoch 79/200
271/271 [==============================] - 0s 734us/step - loss: 28.3928 - val_loss: 35.6066
Epoch 80/200
271/271 [==============================] - 0s 697us/step - loss: 29.1012 - val_loss: 36.1429
Epoch 81/200
271/271 [==============================] - 0s 749us/step - loss: 28.6560 - val_loss: 35.5072
Epoch 82/200
271/271 [==============================] - 0s 804us/step - loss: 28.8311 - val_loss: 36.2556
Epoch 83/200
271/271 [==============================] - 0s 811us/step - loss: 28.5464 - val_loss: 35.2115
Epoch 84/200
271/271 [==============================] - 0s 826us/step - loss: 28.0626 - val_loss: 37.0662
Epoch 85/200
271/271 [==============================] - 0s 767us/step - loss: 28.5353 - val_loss: 35.1223
Epoch 86/200
271/271 [==============================] - 0s 612us/step - loss: 28.2699 - val_loss: 36.5861
Epoch 87/200
271/271 [========================

Epoch 155/200
271/271 [==============================] - 0s 679us/step - loss: 28.0506 - val_loss: 35.0608
Epoch 156/200
271/271 [==============================] - 0s 708us/step - loss: 28.4803 - val_loss: 34.4899
Epoch 157/200
271/271 [==============================] - 0s 752us/step - loss: 27.7605 - val_loss: 35.5914
Epoch 158/200
271/271 [==============================] - 0s 612us/step - loss: 27.6416 - val_loss: 34.4462
Epoch 159/200
271/271 [==============================] - 0s 749us/step - loss: 28.2755 - val_loss: 34.8035
Epoch 160/200
271/271 [==============================] - 0s 789us/step - loss: 28.0464 - val_loss: 34.8777
Epoch 161/200
271/271 [==============================] - 0s 786us/step - loss: 27.8735 - val_loss: 34.5049
Epoch 162/200
271/271 [==============================] - 0s 800us/step - loss: 27.5493 - val_loss: 34.6863
Epoch 163/200
271/271 [==============================] - 0s 822us/step - loss: 28.2082 - val_loss: 35.1853
Epoch 164/200
271/271 [==============

In [22]:
preds = model.predict(testX)

In [30]:
# make predictions on the testing data
preds = model.predict(testX)
 
# compute the difference between the *predicted* house prices and the
# *actual* house prices, then compute the percentage difference and
# the absolute percentage difference
diff = preds.flatten() - testY
percentDiff = (diff / testY) * 100
absPercentDiff = np.abs(percentDiff)
 
# compute the mean and standard deviation of the absolute percentage
# difference
mean = np.mean(absPercentDiff)
std = np.std(absPercentDiff)
 
# finally, show some statistics on our model
locale.setlocale(locale.LC_ALL, "en_US.UTF-8")
print("avg. house price: {}, std house price: {}".format(
    locale.currency(df["price"].mean(), grouping=True),
    locale.currency(df["price"].std(), grouping=True)))
print("mean: {:.2f}%, std: {:.2f}%".format(mean, std))

avg. house price: $533,388.27, std house price: $493,403.08
mean: 34.35%, std: 34.27%


<hr>

## <div style="direction:rtl;text-align:right;font-family:B Lotus, B Nazanin, Tahoma">‌منبع</div>   

https://www.pyimagesearch.com/2019/01/21/regression-with-keras/

<div class="alert alert-block alert-info">
<div style="direction:rtl;text-align:right;font-family:B Lotus, B Nazanin, Tahoma"> دوره مقدماتی یادگیری عمیق<br>علیرضا اخوان پور<br>پنج شنبه، ۲۵ بهمن ۱۳۹۷<br>
</div>
<a href="http://class.vision">Class.Vision</a> - <a href="http://AkhavanPour.ir">AkhavanPour.ir</a> - <a href="https://github.com/Alireza-Akhavan/">GitHub</a>

</div>